### In this notebook Instead of dummy, I apply OneHotEncoder to see if another pre-processing method can lead to better result

### Goal is to be able to predicted count using the given parameters

In [1]:
import datetime 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer


from sklearn.linear_model import LinearRegression

    
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Reading the data
df=pd.read_csv('train.csv')
df.head(10)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1
5,2011-01-01 05:00:00,1,0,0,2,9.84,12.880,75,6.0032,0,1,1
6,2011-01-01 06:00:00,1,0,0,1,9.02,13.635,80,0.0000,2,0,2
7,2011-01-01 07:00:00,1,0,0,1,8.20,12.880,86,0.0000,1,2,3
8,2011-01-01 08:00:00,1,0,0,1,9.84,14.395,75,0.0000,1,7,8
9,2011-01-01 09:00:00,1,0,0,1,13.12,17.425,76,0.0000,8,6,14


In [ ]:
df.info()

I skip the EDA in this notebook

## Split datetime column with converting datetime object

In [3]:
#Pandas to_datetime() method helps to convert string Date time into Python Date time object.
df['datetime']=pd.to_datetime(df['datetime'])
df['datetime']

0       2011-01-01 00:00:00
1       2011-01-01 01:00:00
2       2011-01-01 02:00:00
3       2011-01-01 03:00:00
4       2011-01-01 04:00:00
                ...        
10881   2012-12-19 19:00:00
10882   2012-12-19 20:00:00
10883   2012-12-19 21:00:00
10884   2012-12-19 22:00:00
10885   2012-12-19 23:00:00
Name: datetime, Length: 10886, dtype: datetime64[ns]

In [4]:
#create several dataframe to see if these new feature can specificly affect on count
df['hour']=df['datetime'].dt.hour
df['day']=df['datetime'].dt.day
df['month']=df['datetime'].dt.month
#df['dayofweek']=df['datetime'].dt.dayofweek
df['year']=df['datetime'].dt.year
df

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,hour,day,month,year
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16,0,1,1,2011
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40,1,1,1,2011
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32,2,1,1,2011
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13,3,1,1,2011
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1,4,1,1,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,7,329,336,19,19,12,2012
10882,2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,10,231,241,20,19,12,2012
10883,2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,4,164,168,21,19,12,2012
10884,2012-12-19 22:00:00,4,0,1,1,13.94,17.425,61,6.0032,12,117,129,22,19,12,2012


In [5]:
df=df.set_index(df['datetime'])
df=df.drop(['datetime'],axis=1)  

In [6]:
df

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,hour,day,month,year
datetime,,,,,,,,,,,,,,,
2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16,0,1,1,2011
2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40,1,1,1,2011
2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32,2,1,1,2011
2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13,3,1,1,2011
2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1,4,1,1,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,7,329,336,19,19,12,2012
2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,10,231,241,20,19,12,2012
2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,4,164,168,21,19,12,2012


In [ ]:
df.info()

### Which features most contribute to the number of renting over the specific time?

# Correlation between each feature

In [ ]:
plt.figure(figsize=(12,12))
#correlation
sns.heatmap(df.corr(),vmin=-1,vmax=1,cmap='viridis',annot=True,square=True) #cmap-> color map what color scheme you want to see

#### Based on the heatmap, hour, temp and atemp seems to be the best features that contribute to the count

## Check other features that have impact

Weather seems play a role-> Clear weather more renting and rainy day-> less renting

#temp with atemp are highly co-realted , so we will drop the absolute value i.e., we observed that it has no logic correlation, it has only numeric correlation

#df=df.drop(['atemp'],axis=1)

## Define X,y

In [7]:
X=df.drop(['count'],axis=1)
X

#y=np.log1p(df['count']) 

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,hour,day,month,year
datetime,,,,,,,,,,,,,,
2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,0,1,1,2011
2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,1,1,1,2011
2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,2,1,1,2011
2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,3,1,1,2011
2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,4,1,1,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,7,329,19,19,12,2012
2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,10,231,20,19,12,2012
2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,4,164,21,19,12,2012


In [8]:
y=df['count'] 
y

datetime
2011-01-01 00:00:00     16
2011-01-01 01:00:00     40
2011-01-01 02:00:00     32
2011-01-01 03:00:00     13
2011-01-01 04:00:00      1
                      ... 
2012-12-19 19:00:00    336
2012-12-19 20:00:00    241
2012-12-19 21:00:00    168
2012-12-19 22:00:00    129
2012-12-19 23:00:00     88
Name: count, Length: 10886, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split

In [62]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [63]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8708, 14), (2178, 14), (8708,), (2178,))

# Feature Engineering

In [ ]:
# A breif Explanation of below functions for pre-processing

### ColumnTransformer ->Apply diffeent preprocessing to different columns 
### polynaomial feature ->  improve the relationship between input variables by increasing the signal
### OneHotEncoder -> encode categorical to the numerical one
### make_pipline -> make pipline,the order of execution

categorical_pipe = make_pipeline(OneHotEncoder(sparse=False, handle_unknown='ignore') )

numerical_pipe = PolynomialFeatures(degree= 3, include_bias= False) 

PolynomialFeatures(interaction_only=True, include_bias= False)

feature = make_pipeline(MinMaxScaler() )

# Apply preprocessing on different dataset

trans = ColumnTransformer([
    ('no_extra_info','drop',['casual','registered']),
    ('correlated_with_others','drop',['atemp','season']),
    ('do_nothing','passthrough',['holiday','workingday']),
    ('cat_preprocessing',categorical_pipe,['weather','year','month','dayofweek','hour']),
    ('numeric_preprocessing',numerical_pipe, ['temp','windspeed','humidity']),
    ('interaction',feature,['humidity','temp','windspeed','weather'])
])

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer


cat_pipe = make_pipeline(
OneHotEncoder(sparse=False, handle_unknown='ignore')
)

num_pipe = make_pipeline(MinMaxScaler()
)
    
feat = PolynomialFeatures(degree= 3, include_bias= False)
PolynomialFeatures(interaction_only=True, include_bias= False)

In [64]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer


cat_pipe = make_pipeline(
OneHotEncoder(sparse=False, handle_unknown='ignore')
)

num_pipe = make_pipeline(MinMaxScaler()
)
    
feat = PolynomialFeatures(degree= 3, include_bias= False)
PolynomialFeatures(interaction_only=True, include_bias= False)

PolynomialFeatures(include_bias=False, interaction_only=True)

In [27]:
"""
feature_transformer = ColumnTransformer([
     ( 'correlated_with_others', 'drop', ['atemp', 'season']),
     ( 'not_needed', 'drop', ['casual', 'registered']),
    ( 'do_nothing', 'passthrough', ['holiday', 'workingday']),
    ('cat_preprocessing', cat_pipe, ['weather','year', 'month', 'hour','day']), 
    ('num_preprocessing', num_pipe, ['temp','windspeed', 'humidity' ]),
     ('interaction', feat, ['humidity', 'temp', 'windspeed', 'weather'])
    
    
])
"""



"\nfeature_transformer = ColumnTransformer([\n     ( 'correlated_with_others', 'drop', ['atemp', 'season']),\n     ( 'not_needed', 'drop', ['casual', 'registered']),\n    ( 'do_nothing', 'passthrough', ['holiday', 'workingday']),\n    ('cat_preprocessing', cat_pipe, ['weather','year', 'month', 'hour','day']), \n    ('num_preprocessing', num_pipe, ['temp','windspeed', 'humidity' ]),\n     ('interaction', feat, ['humidity', 'temp', 'windspeed', 'weather'])\n    \n    \n])\n"

In [65]:
feature_transformer = ColumnTransformer([
     ( 'correlated_with_others', 'drop', ['atemp', 'season']),
     ( 'not_needed', 'drop', ['casual', 'registered']),
    ( 'do_nothing', 'passthrough', ['holiday', 'workingday']),
    ('cat_preprocessing', cat_pipe, ['weather','year', 'month', 'day', 'hour']), 
    ('num_preprocessing', num_pipe, ['temp','windspeed', 'humidity' ]),
     ('interaction', feat, ['humidity', 'temp', 'windspeed', 'weather'])
    
    
])

In [66]:
X_train_trans = feature_transformer.fit_transform(X_train)


In [67]:
X_test_trans = feature_transformer.transform(X_test)

In [68]:
from sklearn.linear_model import LinearRegression

In [69]:
LR = LinearRegression()

In [70]:
#train the model
LR.fit(X_train_trans, y_train)


LinearRegression()

In [ ]:
#Intercept --> b0
#print(LR.intercept_)

In [ ]:
# Coefficient -> slope
#print(LR.coef_)

In [ ]:
#predictions=LR.predict(X_test_trans)

In [60]:
round(LR.score(X_train_trans, y_train),2)

0.71

In [71]:
round(LR.score(X_test_trans, y_test),2)

0.72

#plotting y_test and y-pred to understand the spread
fig=plt.figure()
plt.scatter(y_test,predictions)

plt.title('y_test vs predictions',fontsize=20)
plt.xlabel('y_test',fontsize=20)
plt.ylabel('predictions',fontsize=20)